In [ ]:
import os
import torch
import shutil
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Define class mapping
class_names = {0: "no_fire", 1: "fire"}

# Data transformations
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
}


# Load datasets
data_dir = "dl2425_challenge_dataset"

# Jupyter notbooks may create invisible files. Delete them !!!
def remove_ipynb_checkpoints(root_dir):
    for subdir, dirs, files in os.walk(root_dir):
        for dir_name in dirs:
            if dir_name.startswith('.ipynb_checkpoints'):
                dir_path = os.path.join(subdir, dir_name)
                shutil.rmtree(dir_path)
                print(f"Removed folder: {dir_path}")

# Apply to dataset folder
remove_ipynb_checkpoints(data_dir)

# Preprocess the datasets
train_dataset =  datasets.ImageFolder(root='dl2425_challenge_dataset/train', transform=data_transforms["train"])
val_dataset =  datasets.ImageFolder(root='dl2425_challenge_dataset/val', transform=data_transforms["val"])


# Data loaders
batch_size = 32
dataloaders = {
    "train": DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
    "val": DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
}

# Dataset sizes
dataset_sizes = {
    "train": len(train_dataset),
    "val": len(val_dataset)
}

print(f"Classes: {class_names}")
print(f"Dataset Sizes: {dataset_sizes}")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from torchvision import models
import torch.nn as nn

# Load pre-trained ResNet
model = models.resnet18(pretrained=True)

# Modify the final layer for binary classification
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)  # Binary classification requires 1 output (logit)

model = model.to(device)
print('Model setup: done')

In [ ]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()  # Combines Sigmoid activation and BCELoss
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)  # Reduce LR every 7 epochs
print('Hyperparameters setup: done')

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    loss_history = {'train': [], 'val': []}
    acc_history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                # Binary classification labels need to be float (0.0 or 1.0)
                labels = labels.float().unsqueeze(1)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    preds = torch.sigmoid(outputs) > 0.5  # Convert logits to binary predictions

                    # Backward + optimize only in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # Store metrics
            loss_history[phase].append(epoch_loss)
            acc_history[phase].append(epoch_acc.item())
            
            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    print(f"Best val Acc: {best_acc:4f}")
    model.load_state_dict(best_model_wts)

    # Plot loss and accuracy
    epochs_range = range(1, num_epochs + 1)

    plt.figure(figsize=(12, 5))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, loss_history['train'], label='Training Loss')
    plt.plot(epochs_range, loss_history['val'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, acc_history['train'], label='Training Accuracy')
    plt.plot(epochs_range, acc_history['val'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()
    return model
    


In [ ]:
num_epochs = 25
model = train_model(model, criterion, optimizer, scheduler, num_epochs)


In [ ]:
torch.save(model.state_dict(), 'best_fire_classifier.pth')
print("The model has been saved")